# Capstone Project - The Battle of the Neighborhoods (Week 1 and 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

Healing-Touch is a famous pharmaceutical company that wish to establish a 'Medical Supply Store' somewhere in Bronx Area. In this project we will try to find an optimal location for a 'Medical Supply'. Specifically, this report will be targeted to Healing-Touch Pharmaceuticals.

Bronx may have some 'Medical Supply stores' in some neighbourhoods already. So our target would be to get an overview about the locations having lack of Medical Supply stores.

We will use  data science to generate a few most  neighborhoods based on this criteria and suggestions can be made to client based on our analysis

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* presence of existing Medical Supplies in the neighborhood


Following data sources will be needed to extract/generate the required information:
* Coordinate of Bronx, NY neighburhood can be extracted from the available newyork json data
* Coordinate of Bronx, NY city centre can be generated by using Google Maps API geocoding
* All kind of Veneues in every neighborhood will be obtained using **Foursquare API**


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.8-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.5.30          |   py38h578d9bd_0         141 KB  conda-forge
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    openssl-1.1.1k             |       h7f98852_0         2.1 MB  conda-forge
    python_abi-3.8             |           2_cp38           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0
  pyth

## 1. Download and Explore Dataset


In [4]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [6]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [9]:
neighborhoods_data = newyork_data['features']

In [17]:

neighborhoods_data[0]


{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a _pandas_ dataframe

In [10]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Then let's loop through the data and fill the dataframe one row at a time.

In [11]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [38]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [39]:
Bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
Bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Let's first find the latitude & longitude of Bronx city center, using specific, well known address and Google Maps geocoding API.

In [40]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bronx are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bronx are 40.8466508, -73.8785937.


Let's visualizat Bronx with the neighborhoods in it.

In [41]:
# create map of Bronx using latitude and longitude values
map_Bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Bronx_data['Latitude'], Bronx_data['Longitude'], Bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Bronx)  
    
map_Bronx

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on Venuews in each neighborhood.

We're interested in venues named 'Medical Supplies'

#### Define Foursquare Credentials and Version

In [14]:
CLIENT_ID = 'MU1KAFNUNAZUM4WHF5LJVFEOGJUXIF1YCC30PVQGMCOIIIOG' # your Foursquare ID
CLIENT_SECRET = 'ZSPILD0X5KX3DOUKKD514DBLAXZYB203GZXYVFVGLMOCPX2N' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MU1KAFNUNAZUM4WHF5LJVFEOGJUXIF1YCC30PVQGMCOIIIOG
CLIENT_SECRET:ZSPILD0X5KX3DOUKKD514DBLAXZYB203GZXYVFVGLMOCPX2N


## 2. Explore Neighborhoods in Bronx


In [65]:
#### Let's create a function to repeat the same process to all the neighborhoods in Bronx

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _Bronx_venues_.

In [42]:
Bronx_venues = getNearbyVenues(names=Bronx_data['Neighborhood'],
                                   latitudes=Bronx_data['Latitude'],
                                   longitudes=Bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


#### Let's check the size of the resulting dataframe

In [43]:
print(Bronx_venues.shape)
Bronx_venues.head()

(1234, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


Let's check how many venues were returned for each neighborhood

In [44]:
Bronx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,32,32,32,32,32,32
Baychester,24,24,24,24,24,24
Bedford Park,31,31,31,31,31,31
Belmont,100,100,100,100,100,100
Bronxdale,15,15,15,15,15,15
Castle Hill,7,7,7,7,7,7
City Island,29,29,29,29,29,29
Claremont Village,22,22,22,22,22,22
Clason Point,13,13,13,13,13,13


#### Let's find out how many unique categories can be curated from all the returned venues


In [45]:
print('There are {} uniques categories.'.format(len(Bronx_venues['Venue Category'].unique())))

There are 167 uniques categories.


## 3. Analyze Each Neighborhood

In [46]:
# one hot encoding
Bronx_onehot = pd.get_dummies(Bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Bronx_onehot['Neighborhood'] = Bronx_venues['Neighborhood'] 



In [42]:
# move neighborhood column to the first column
fixed_columns = [Bronx_onehot.columns[-1]] + list(Bronx_onehot.columns[:-1])
Bronx_onehot = Bronx_onehot[fixed_columns]

Bronx_onehot.head()

,Neighborhood,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping 

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [47]:
Bronx_grouped = Bronx_onehot.groupby('Neighborhood').mean().reset_index()
Bronx_grouped

,Neighborhood,ATM,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Liquor Store,Lounge,Market,Martial Arts School,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Nightclub,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Post Office,Pub,Puerto Rican Restaurant,Recording Studio,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sports Bar,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Allerton,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.00,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.093750,0.031250,0.031250,0.000000,0.062500,0.000000,0.00000,0.00000,0.031250,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.031250,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.00,0.093750,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00,0.000000,0.031250,0.031250,0.000000,0.000000,0.093750,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Baychester,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.0

#### Let's put that into a _pandas_ dataframe

First, let's write a function to sort the venues in ascending order.

In [67]:
def return_least_common_venues(row, num_bottom_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=True)
    
    return row_categories_sorted.index.values[0:num_bottom_venues]

Now let's create the new dataframe and display the least occuring 10 venues for each neighborhood.

In [68]:
num_bottom_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of bottom venues
columns = ['Neighborhood']
for ind in np.arange(num_bottom_venues):
    try:
        columns.append('{}{} Least Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Least Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Bronx_grouped['Neighborhood']

for ind in np.arange(Bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_least_common_venues(Bronx_grouped.iloc[ind, :], num_bottom_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,10th Least Common Venue
0,Allerton,ATM,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mattress Store,Moving Target
1,Baychester,ATM,Medical Supply Store,Mediterranean Restaurant,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Nightclub,Outdoors & Recreation
2,Bedford Park,ATM,Market,Martial Arts School,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Miscellaneous Shop,Mobile Phone Shop
3,Belmont,ATM,Mattress Store,Medical Supply Store,Men's Store,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Lounge,Moving Target,Nightclub
4,Bronxdale,ATM,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Mattress Store,Music Venue
5,Castle Hill,ATM,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Mediterranean Restaurant,Nightclub
6,City Island,ATM,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Nightclub,Outdoors & Recreation
7,Claremont Village,ATM,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Mediterranean Restaurant,Nightclub
8,Clason Point,ATM,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Martial Arts School,Mobile Phone Shop
9,Co-op City,ATM,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mattress Store,Moving Target


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.

In [69]:
# set number of clusters
kclusters = 5

Bronx_grouped_clustering = Bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 4, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 3, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the bottom 10 venues for each neighborhood.

In [70]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Bronx_merged = Bronx_data

# merge Bronx_grouped with Bronx_data to add latitude/longitude for each neighborhood
Bronx_merged = Bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Bronx_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,10th Least Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0,ATM,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Nightclub,Men's Store,Outdoors & Recreation
1,Bronx,Co-op City,40.874294,-73.829939,0,ATM,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mattress Store,Moving Target
2,Bronx,Eastchester,40.887556,-73.827806,0,ATM,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Mediterranean Restaurant,Nightclub
3,Bronx,Fieldston,40.895437,-73.905643,4,ATM,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Nightclub,Men's Store,Outdoors & Recreation
4,Bronx,Riverdale,40.890834,-73.912585,0,ATM,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Mediterranean Restaurant,Nightclub


Finally, let's visualize the resulting clusters

In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Bronx_merged['Latitude'], Bronx_merged['Longitude'], Bronx_merged['Neighborhood'], Bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Bronx that have low Medical Supplies density.

In first step we have collected the least occuring venues for all the neighbourhoods in Bronx, NY

Second step in our analysis will be to find out the neighbourhoods having least frequency of Medical Supplies

In third and final step we will focus on most promising areas for exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's find out the neighbourhoods having Medical Supplies as the least occuring venue in bottom 2

In [87]:
Bronx_Medical=Bronx_merged.iloc[:,0:7].drop(['Cluster Labels'],axis=1)
Bronx_Medical

,Borough,Neighborhood,Latitude,Longitude,1st Least Common Venue,2nd Least Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,ATM,Metro Station
1,Bronx,Co-op City,40.874294,-73.829939,ATM,Medical Supply Store
2,Bronx,Eastchester,40.887556,-73.827806,ATM,Men's Store
3,Bronx,Fieldston,40.895437,-73.905643,ATM,Metro Station
4,Bronx,Riverdale,40.890834,-73.912585,ATM,Men's Store
5,Bronx,Kingsbridge,40.881687,-73.902818,ATM,Laundromat
6,Bronx,Woodlawn,40.898273,-73.867315,ATM,Mattress Store
7,Bronx,Norwood,40.877224,-73.879391,ATM,Mattress Store
8,Bronx,Williamsbridge,40.881039,-73.857446,ATM,Men's Store
9,Bronx,Baychester,40.866858,-73.835798,ATM,Medical Supply Store


We shall include only the neighbourhoods  having 'Medical Supply Store' in any above

In [171]:
Bronx_Medical1 = Bronx_Medical[Bronx_Medical['2nd Least Common Venue']=='Medical Supply Store']

In [174]:
Bronx_Medical1

,Borough,Neighborhood,Latitude,Longitude,1st Least Common Venue,2nd Least Common Venue
1,Bronx,Co-op City,40.874294,-73.829939,ATM,Medical Supply Store
9,Bronx,Baychester,40.866858,-73.835798,ATM,Medical Supply Store
10,Bronx,Pelham Parkway,40.857413,-73.854756,ATM,Medical Supply Store
18,Bronx,High Bridge,40.836623,-73.926102,ATM,Medical Supply Store
20,Bronx,Mott Haven,40.806239,-73.916100,ATM,Medical Supply Store
21,Bronx,Port Morris,40.801664,-73.913221,ATM,Medical Supply Store
23,Bronx,Hunts Point,40.809730,-73.883315,ATM,Medical Supply Store
35,Bronx,North Riverdale,40.908543,-73.904531,ATM,Medical Supply Store
36,Bronx,Pelham Bay,40.850641,-73.832074,ATM,Medical Supply Store
49,Bronx,Bronxdale,40.852723,-73.861726,ATM,Medical Supply Store


In [91]:
!pip install haversine

In [172]:
import haversine as hs # for calculating the distance betweeen 2 points with Coordinates available



Calculate the distance of each neighbourhood from Bronx City Centre

In [169]:
print('--------------------Distance from City Centre-------------------------\n')
print('------Neighbourhood------------------------Distance from Bronx Centre (kms)------\n')

for t in range(len(Bronx_Medical1)):
    latsrt=Bronx_Medical1.iloc[t, 2]
    lonsrt=Bronx_Medical1.iloc[t, 3]
    loc2= (latsrt,lonsrt)
    distan=round(hs.haversine(loc2,Bronx_loc),2)
    print(' ' * 7 , Bronx_Medical1.iloc[t, 1], '  ' * 20 ,  distan)
    


--------------------Distance from City Centre-------------------------

------Neighbourhood------------------------Distance from Bronx Centre (kms)------

        Co-op City                                          5.12
        Baychester                                          4.24
        Pelham Parkway                                          2.33
        High  Bridge                                          4.15
        Mott Haven                                          5.49
        Port Morris                                          5.79
        Hunts Point                                          4.12
        North Riverdale                                          7.22
        Pelham Bay                                          3.94
        Bronxdale                                          1.57
        Allerton                                          2.68


## Results and Discussion <a name="results"></a>

Our analysis shows that although there are quite many neighbourhoods in Bronx where lack of Medical Supply can be found, we have shortlisted the list where 'Medical Supply' is the least found venue.

After directing our attention to this more narrow area of interest , we found out the distance between the shortlisted neighbourhoods and the Bronx city centre.

Result of all this is 11 neighbourhoods with the distance from city centre ranging between 1.5 to 6 kms. Client can now be provided with these options for creating a 'Medical Supply Centre'

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Bronx neighbourhood close to center with low number of 'Medical Supply Centre'in order to aid stakeholders in narrowing down the search for optimal location. By calculating 'Medical Supply density' density distribution from Foursquare data we have first identified neighbourhood that justify further analysis by stakeholders.

Final decission on optimal 'Medical Supply Centre' location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like social and economic dynamics of every neighborhood, convenience of transportation, etc.